# Student: McLaughlin, Chris

# Problem 3. Pigs, begone!

## Notes/README
- Notebook is heavy on memory usage and MUST be run in an environment with sufficient RAM - Assignment was done in Google Collab!
- Ensure that enron5.tar.gz is present in the working directory!

## Acknowledgements/Citations

- Lecture Slides - 12 - Bayes
- https://en.wikipedia.org/wiki/Naive_Bayes_classifier
- https://jonndata.github.io/2020-07-30-Naive-Bayes-From-Scratch/
- https://medium.com/@rangavamsi5/na%C3%AFve-bayes-algorithm-implementation-from-scratch-in-python-7b2cc39268b9
- https://stackoverflow.com/questions/18617244/read-contents-of-tar-gz-file-from-website-into-a-python-3-x-object
- https://stackoverflow.com/questions/26695903/python-how-to-read-all-files-in-a-directory
- https://www.geeksforgeeks.org/python-dictionary-keys-method/#:~:text=The%20keys()%20method%20in,order%20of%20insertion%20using%20Python.&text=Parameters%3A%20There%20are%20no%20parameters,that%20displays%20all%20the%20keys.
- https://www.w3schools.com/python/python_howto_remove_duplicates.asp

## Setup

In [ ]:
# This should be roughly the content of the first code cell
import numpy as np
import random
np.random.seed(1337)
random.seed(1337)
# for downloading and importing the data
import tarfile
from pathlib import Path

In [ ]:
# Plotting support
from matplotlib import pyplot as plt
#from plotnine import *
# Standard libraries
import pandas as pd
#import sklearn as sk

## Some theory before we do this

Bayesian Problem (from wikipedia and lecture slides):

$ \frac{P(spam|x_i)P(spam)}{P(x_i)} = \frac{prior \times liklihood}{evidence} $ 

where the denominator is irrelevant.

The Naïve Bayes formula then, after applying MLE, is:

$ P(\text{spam}|x_i) \propto P(x_i|\text{spam})P(\text{spam}) = (\frac{\text{number of spam emails}}{\text{number of messages}}) ∏_{j}(\frac{\text{number of spam emails containing word j}}{\text{number of spam messages}}) $

ie. Multiply the observed probabilities of each word being present in spam emails together, and then multiply this by the probability of an email being spam in the first place. The resulting probability is proportional to $ P(spam|x_i) $, which is good enough for our purposes as we're going to construct a classifier which simply selects the class with the highest probability.

## Data Import and Processing

We're going to split our data into 72% train and 28% test (needed for nice numbers on the split!), and represent each email as a bag of words

In [ ]:
# Extract the tar
tar = tarfile.open("enron5.tar.gz")
tar.extractall("./temp_assignment2_files")
tar.close

<bound method TarFile.close of <tarfile.TarFile object at 0x7faca1e56510>>

In [ ]:
# Import files to two seperate lists
spam_emails=[]
ham_emails=[]
for thing in Path("./temp_assignment2_files/enron5/spam").iterdir():
  if thing.is_file():
    spam_emails.append(thing.read_text(encoding="latin-1"))
for thing in Path("./temp_assignment2_files/enron5/ham").iterdir():
  if thing.is_file():
    ham_emails.append(thing.read_text(encoding="latin-1"))

### Data Splitting and Model Training

In [ ]:
# Do the test/train split
sdivider = int(len(spam_emails)*0.72)
hdivider = int(len(ham_emails)*0.72)

#print(sdivider)
#print(hdivider)

spam_emails_train = spam_emails[:sdivider]
spam_emails_test = spam_emails[sdivider:]
ham_emails_train = ham_emails[:hdivider]
ham_emails_test = ham_emails[hdivider:]

#print(len(spam_emails_train))
#print(len(spam_emails_test))
#print(len(ham_emails_train))
#print(len(ham_emails_test))

# Convert emails to lists of words
for index in range(len(spam_emails_train)):
  spam_emails_train[index] = spam_emails_train[index].split()
for index in range(len(ham_emails_train)):
  ham_emails_train[index] = ham_emails_train[index].split()

# Figure out our observed probabilities
spamdict={}
hamdict={}

for email in spam_emails_train:
  for word in email:
    spamdict[word]=0
for email in ham_emails_train:
  for word in email:
    hamdict[word]=0

# Train the model (ie. Learn the number of spam emails and ham emails per word)
""" this was O(n^absolutelynot) so we're going to try something else
for key in spamdict.keys():
  for email in spam_emails_train:
    if key in email:
      spamdict[key]+=1
for key in hamdict.keys():
  for email in ham_emails_train:
    if key in email:
      hamdict[key]+=1
"""

def list_unique_vals(inlist): #It's super stupid that python doesn't have a builtin list method to do this - this is from the w3schools article in the acknowledgements
  return list(dict.fromkeys(inlist))

for index in range(len(spam_emails_train)):
  spam_emails_train[index] = list_unique_vals(spam_emails_train[index])
for index in range(len(ham_emails_train)):
  ham_emails_train[index] = list_unique_vals(ham_emails_train[index])

for email in spam_emails_train:
  for word in email:
    spamdict[word]+=1
for email in ham_emails_train:
  for word in email:
    hamdict[word]+=1

# Lets get our P(spam) and P(ham)
Pspam = len(spam_emails_train)/(len(spam_emails_train)+len(ham_emails_train))
Pham = len(ham_emails_train)/(len(ham_emails_train)+len(spam_emails_train))

# One last thing - lets convert our dicts to probabilities rather than absolute values - this will save us having to do it in the classifier
for key in spamdict.keys():
  spamdict[key] = spamdict[key]/len(spam_emails_train)
for key in hamdict.keys():
  hamdict[key] = hamdict[key]/len(ham_emails_train)

The model is now trained! We have dictionaries for each class composed of the number of emails of that class that contain a given word, as well as Pspam. That's actually all we need to start constructing our actual classifier!

### Classifier

We'll construct our classifier function here, and then do the actual predictions later!

In [ ]:
def isSpam(email):
  email = email.split()
  email = list_unique_vals(email)
  
  spam_probs = []
  ham_probs = []

  # Get spam probs
  for word in email:
    if word in spamdict:
      spam_probs.append(spamdict[word])
    else:
      spam_probs.append(0) # Note: This is a TERRIBLE way of doing this for reasons I'll discuss in proper markup text later on - but it's by the book, the lecture slides show this being the algorithm
  
  # Get ham probs
  for word in email:
    if word in hamdict:
      ham_probs.append(hamdict[word])
    else:
      ham_probs.append(0)

  # We now have our collection of probabilities (x_i|spam) and (x_i|ham)

  # Okay, let's work out whether this email is more likely to be spam or ham
  # P(spam|x_i)
  Pspamx = Pspam
  for prob in spam_probs:
    Pspamx *= prob
  # P(ham|x_i)
  Phamx = Pham
  for prob in ham_probs:
    Phamx *= prob
  
  # Remember these are not "true" probabilities, they're just proportional to the true probability. This is fine though, it just means rather than 1-Pspam necessarily equalling Pham, we just go with whichever of Pspam and Pham is larger.
  # Get prediction
  if Pspamx > Phamx:  # Ties break for ham, because assignment doesn't say which way to do it and I feel like IRL you'd want to see the email if it was a tossup
    return "SPAM" # We could be clean and use a boolean here but I feel like it'll make later code easier to maintain non-confusing if-equals-ability :D
  else:
    return "HAM"

Okay a quick followup and justification on that comment and why assigning a probability of 0 to words not in the dictionary is such a bad idea, and why I just did it anyway:

 Since all our $ P(x_i|spam) $ and $ P(x_i|ham) $ probabilities for each word $ i $ in the email are getting multiplied together, any 0s in there are going to COMPLETELY defeat the rest of the probabilities, and return a $ P(spam|x_i) $ or $ P(ham|x_i) $ probability of 0, even if it's just one word that isn't in the dictionary! This is obviously a big drag on the model, but it's how it's shown in the lecture slides so it's how I implemented it. It may be mitigated somewhat by the fact that we take our spam and ham probabilities and go with the larger, but that remains to be seen.

### Predictions

Okay let's see if this thing works!

In [ ]:
# Becase this is our from-scratch model, we control what information the model actually looks at, that means we don't need any complicated accuracy measuring with a combined spam and ham dataset like we would have in the last two questions, we can just do a simple version of that ourselves and save the time fussing with numpy.

corrects=0
misses=0

corrects_train=0
misses_train=0

# Testing Accuracy
for email in spam_emails_test:
  result = isSpam(email)
  if result=="SPAM":
    corrects += 1
  elif result == "HAM":
    misses += 1
  else:
    print("ERROR: Something is *very* messed up here!")

for email in ham_emails_test:
  result = isSpam(email)
  if result=="HAM":
    corrects += 1
  elif result=="SPAM":
    misses+=1
  else:
    print("ERROR: Something is *very* messed up here!")

# Restore our clean training dataset
spam_emails_train = spam_emails[:sdivider]
ham_emails_train = ham_emails[:hdivider]

# Training Accuracy
for email in spam_emails_train:
  result = isSpam(email)
  if result=="SPAM":
    corrects_train += 1
  elif result == "HAM":
    misses_train += 1
  else:
    print("ERROR: Something is *very* messed up here!")

for email in ham_emails_train:
  result = isSpam(email)
  if result=="HAM":
    corrects_train += 1
  elif result=="SPAM":
    misses_train+=1
  else:
    print("ERROR: Something is *very* messed up here!")

# Sanity check!
if (corrects+misses+corrects_train+misses_train) != (len(spam_emails_test)+len(spam_emails_train)+len(ham_emails_test)+len(ham_emails_train)):
  print("ERROR: We appear to have lost some? :D")

# Do math and output results
training_accuracy = corrects_train/(corrects_train+misses_train)
testing_accuracy = corrects/(corrects+misses)

print("Training Accuracy:", training_accuracy,"\nTesting Accuracy:", testing_accuracy)

Training Accuracy: 0.9119699409554483 
Testing Accuracy: 0.4492753623188406


Okaaaay! So we have solidly decent training accuracy, while testing accuracy is literally worse than if we had just flipped a coin.

So the model isn't great. Right out of the gate this looks like overfitting, which makes sense given our model literally does not have the ability to handle words that didn't come up in the training data, and we didn't have all *that* many emails in our dataset to begin with.

The fact that we use a product for our probabilities and consider any words that don't come up in the relevant dictionary as $ P(x_i) = 0 $ probably isn't helping here - I *did* get curious and google this, apparently there's a technique called fuzzing that counteracts the negative effects of that, but that's a project for another time!